<a href="https://colab.research.google.com/github/narhim/nn_project/blob/master/xlnet_pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pwd

'/content/drive/My Drive'

In [ ]:
!ls

In [ ]:
%cd /content/drive/MyDrive/NN_groupWS2021/final_project/
%pwd

/content/drive/.shortcut-targets-by-id/1JjKm5t8lyp2F4kr-Bq8HIArzsrnahAhe/NN_groupWS2021/final_project


'/content/drive/.shortcut-targets-by-id/1JjKm5t8lyp2F4kr-Bq8HIArzsrnahAhe/NN_groupWS2021/final_project'

In [ ]:
import csv

In [ ]:
sent_word_ids = []
sents = []
sents_pos = []
num_sents = 0
with open('sample.tsv') as file:
    #tsv_file = csv.reader(file, delimiter="\t", quotechar='"')
    word_ids = []
    words = []
    pos = []
    for line in file:
        try:
            id, word, tag = line.rstrip('\n').split('\t')
        except ValueError:
            # this will happen when you get to the line where there's only the '*' token
            num_sents += 1
            # reach the * line
            id = line.rstrip('\n').split()[0]
        if id == '*':
            assert len(word_ids) == len(words) == len(pos)
            #print(word_ids)
            #print(words)
            #print(pos)
            sent_word_ids.append(word_ids)
            sents.append(words)
            sents_pos.append(pos)
            word_ids = []
            words = []
            pos = []
        else:
            word_ids.append(int(id))
            words.append(word)
            pos.append(tag)
assert len(sent_word_ids) == len(sents) == len(sents_pos)

In [ ]:
cnt = 0
for w_idx, s, t in zip(sent_word_ids, sents, sents_pos):
    print(w_idx,'\n', s, '\n', t, '\n')
    cnt += 1
    if cnt > 10:
        break

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 
 ['A', 'Saudi', 'woman', 'issues', 'a', 'voice', 'recording', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'and', 'demands', 'the', 'punishment', 'of', 'his', 'killers', '.'] 
 ['DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'WDT', 'PRP', 'VBZ', 'PRP$', 'NN', 'CC', 'VBZ', 'DT', 'NN', 'IN', 'PRP$', 'NNS', '.'] 

[0] 
 ['HAD1999'] 
 ['NNP'] 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56] 
 ['In', 'a', 'unique', 'way', ',', 'a', 'young', 'Saudi', 'woman', 'has', 'resorted', 'to', 'issuing', 'a', 'sound', 'tape', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'murdered', 'in', 'Hafr', 'Al', 'Batin', ',', 'in', 'the', 'hope', 'of', 'rekindling', 'interest', 'in', 'his', 'case', ',', 'in', 'which', '3', 'young', 'men', 'were', 'accused',

## 80/10/10 train, val, test splits

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(sents, sents_pos, test_size=.2, random_state=42)
val_texts, test_texts, val_tags, test_tags = train_test_split(val_texts, val_tags, test_size=.5, random_state=42)

In [ ]:
#Toy samples for faster development
#top = int(0.01*len(sents))
#sents = sents[0:top]
#sents_pos = sents_pos[0:top]
#train_texts, val_texts, train_tags, val_tags = train_test_split(sents, sents_pos, test_size=.2, random_state=42)
#val_texts, test_texts, val_tags, test_tags = train_test_split(val_texts, val_tags, test_size=.5, random_state=42)

## create encodings for our tokens and tags.

In [ ]:
unique_tags = set(tag for doc in sents_pos for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}


## install transformers

In [ ]:
%pip install transformers

     |████████████████████████████████| 2.0MB 16.9MB/s 
     |████████████████████████████████| 3.2MB 56.2MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=44faabbe3e966c475ece5d071de98bfcefa5f16b5501a40a4859835e39d7ab05
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## token encodings for train and validation sets

In [ ]:
#Bert
from transformers import XLNetTokenizerFast
tokenizer = XLNetTokenizerFast.from_pretrained('xlnet-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Bert and many models like it use a method called WordPiece
Tokenization, meaning that single words are split into multiple tokens such that each token is likely to be in the
vocabulary. For example, DistilBert's tokenizer would split the Twitter handle `@huggingface` into the tokens `['@',
'hugging', '##face']`. This is a problem for us because we have exactly one tag per token. If the tokenizer splits a
token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.

One way to handle this is to only train on the tag labels for the first subtoken of a split token. We can do this in
Transformers by setting the labels we wish to ignore to `-100`. In the example above, if the label for
`@HuggingFace` is `3` (indexing `B-corporation`), we would set the labels of `['@', 'hugging', '##face']` to
`[3, -100, -100]`.

For each sub-token returned by the tokenizer, the offset mapping gives us a tuple indicating the sub-token's
start position and end position relative to the original token it was split from. That means that if the first position
in the tuple is anything other than `0`, we will set its corresponding label to `-100`. While we're at it, we can
also set labels to `-100` if the second position of the offset mapping is `0`, since this means it must be a
special token like `[PAD]` or `[CLS]`.

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)
test_labels = encode_tags(test_tags, test_encodings)

ValueError: ignored

## create a dataset object:

In [ ]:
import torch

class OntonetesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
#try:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
test_encodings.pop("offset_mapping")
#except:
#  pass  
train_dataset = OntonetesDataset(xlnet_train_encodings, xlnet_train_labels)
val_dataset = OntonetesDataset(xlnet_val_encodings, xlnet_val_labels)
test_dataset = OntonetesDataset(xlnet_test_encodings, xlnet_test_labels)

## check the dataset

In [ ]:
len(train_dataset)

In [ ]:
train_dataset[0]

## Load model for classification

In [ ]:
from transformers import XLNetBertForTokenClassification
model = XLNetForTokenClassification.from_pretrained('xlnet-base-cased', num_labels=len(unique_tags))

##Install wandb to keep track of training
User: narhim
Password: 123579trms121092
(Delete before submitting)

In [ ]:
%pip install wandb
import wandb
wandb.login()
%env WANDB_PROJECT = nn_project_xlnet
%env WANDB_LOG_MODEL = true

##Training and Validation

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.multioutput import MultiOutputClassifier

In [ ]:

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

#Send to CUDA
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)



#train_loader = DataLoader(train_dataset, batch_size_train, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size_val, shuffle=True)
#optim = AdamW(model.parameters(), lr=5e-5)
#
#for epoch in range(1, n_epochs + 1):
#    train(optim,train_loader,model,epoch)
#    val(val_loader,model,epoch)




In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

#model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)
trainer.train()
trainer.evaluate()

In [ ]:
#Save model
trainer.save_model()
#Make predictions
predictions, label_ids, metrics = trainer.predict(val_dataset)


##Evaluation

In [ ]:
#Evaluation functions
#Evaluation functions
def evaluate(model, test_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for batch in eval_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      tokens = batch['token_type_ids'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      eval_loss = outputs[0]
      print(eval_loss)
      preds = outputs[1].argmax(2, keepdim=False)
      for t in tokens:
        label_words=bert_tokenizer.convert_ids_to_tokens(t)
        print(label_words)
        #if l != "logits" and l!=loss:
        #  for g in l:
        #    label_words=bert_tokenizer.convert_ids_to_tokens(g)
        #    print(label_words)
      print("These are the preds")
      #print(preds)
      #for p in preds:
      #  predicted_token = bert_tokenizer.convert_ids_to_tokens(p)
      #  print("These are the words")
      #  print(predicted_token)
      #print("these are the labels")
      #for l in labels:
      #  label_words=bert_tokenizer.convert_ids_to_tokens(l)
      #  print(label_words)
      #print(labels.shape)
      #print(model.decode(labels))
      #labels.reshape(1)
      #preds.reshape(preds.shape[0])
      #BUG: CALCULATING SCORES, MULTICLASS
      #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
      #acc = accuracy_score(labels, preds)
      #correct += preds.eq(labels.view_as(preds)).sum()
      
    
  eval_loss /= len(eval_loader.dataset)
  eval_losses.append(eval_loss)
  print('\nEvaluation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    eval_loss, correct, len(eval_loader.dataset),
    100. * correct / len(eval_loader.dataset)))
    

In [ ]:
#Make predictions
#eval_losses = []
#model.eval()
#eval_loader = DataLoader(test_dataset, batch_size_val, shuffle=True)
#evaluate(model, eval_loader)

In [ ]:
#for b in eval_loader:
#  print(b)